In [ ]:

abfsPath = 'abfss://dd203327-0824-4acb-9fa0-979973d61328@onelake.dfs.fabric.microsoft.com/b4b8be89-7d54-43f8-b31c-f3ba50d4dfdf/'
pTableName = 'camis.ips_providrepisode__ct'
pJoinKey= 'ipe_epinumber'
pOrderKey= 'header__change_seq'


In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import col,lit,current_timestamp
 
spark.sql("set spark.sql.streaming.schemaInference=true")
 
def loadIncrementals(table_name, join_key, order_key):
 
    def upsertToDeltaCaptureCDC(microBatchOutputDF, batchId):
        targettable = microBatchOutputDF.select("fep_table_name").limit(1).collect()[0][0]
        joinkey = microBatchOutputDF.select("fep_join_key").limit(1).collect()[0][0]
        orderkey = microBatchOutputDF.select("fep_order_key").limit(1).collect()[0][0]
        microBatchOutputDF = microBatchOutputDF.drop("fep_join_key").drop('fep_table_name').withColumn("changeTimestamp",current_timestamp())
        tablecols = spark.sql('show columns in '+targettable.replace('.','_').replace('__ct','')).collect()
        updatestmt = ''
        insertsrcstmt = ''
        inserttgtstmt = ''
        mergestmt = ''
        for column in tablecols:
          #print(column.col_name)
          updatestmt = updatestmt+column.col_name +'= source.'+column.col_name + ','
          insertsrcstmt = insertsrcstmt + column.col_name+ ','
          inserttgtstmt = inserttgtstmt + "source."+column.col_name+ ','
        microBatchOutputDF.createOrReplaceTempView("updates")
        #updatestmt = updatestmt + 'changeTimestamp = source.changeTimestamp'
        #insertsrcstmt =  insertsrcstmt + 'changeTimestamp'
        #inserttgtstmt = inserttgtstmt + 'source.changeTimestamp'
        mergestmt = '''MERGE INTO '''+targettable.replace('.','_').replace('__ct','')+''' target
        USING (SELECT X.* FROM
              (
              SELECT
                  DENSE_RANK() OVER (PARTITION BY CT.'''+joinkey+''' ORDER BY CT.'''+orderkey+''' desc) AS DR
              ,   CT.*
              FROM
                  updates AS CT
              ) X
          WHERE
              1=1
              AND DR = 1
              AND header__change_oper NOT IN ('B','D')
              )  source
        ON source.'''+ joinkey + '''=  target.'''+joinkey+'''
        WHEN MATCHED THEN UPDATE SET ''' + updatestmt.rstrip(',') + '''
        WHEN NOT MATCHED THEN INSERT (''' + insertsrcstmt.rstrip(',') + ''') VALUES ('''+inserttgtstmt.rstrip(',') + ''')'''
        #display(mergestmt)
        microBatchOutputDF.sparkSession.sql(mergestmt)
        
 
 
    streamingUpserts = spark.readStream.format("parquet").load(abfsPath+"Files/Raw_qlik_source/"+table_name+"/").withColumn("fep_table_name", lit(table_name)) \
      .withColumn("fep_join_key", lit(join_key)).withColumn("fep_order_key", lit(order_key))
    #display(streamingUpserts)
 
 
    #Apply to the main-delta-table
    query = streamingUpserts.writeStream \
      .format("delta") \
      .trigger(once=True) \
      .foreachBatch(upsertToDeltaCaptureCDC) \
      .option("checkpointLocation",abfsPath+"Files/Raw_qlik_source/"+table_name+"/checkpoints/") \
      .outputMode("update") \
      .start()
    query.awaitTermination()

#Use only folders that are not empty and end with a name ending in __ct 
folder_path = abfsPath+"Files/Raw_qlik_source/"+pTableName+"/"
if any(mssparkutils.fs.ls(folder_path)) and folder_path.endswith('__ct/'): 
  loadIncrementals(pTableName,pJoinKey,pOrderKey)

In [ ]:
mssparkutils.notebook.exit("Successfully loaded "+ pTableName)

In [ ]:
#print(len(mssparkutils.fs.ls(folder_path)))
